In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [3]:
from datasets import load_dataset

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
dataset=load_dataset("PrabalAryal/nepali-error-and-corrected-sentence-corpus")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/447 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/118M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/50.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/559967 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/239986 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'corrupted'],
        num_rows: 559967
    })
    test: Dataset({
        features: ['sentence', 'corrupted'],
        num_rows: 239986
    })
})

In [7]:
train_dataset=dataset['train'].select(range(5000))
test_dataset=dataset['test'].select(range(2000))

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForMaskedLM


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [9]:

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = TFAutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at google/muril-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [10]:
train_dataset

Dataset({
    features: ['sentence', 'corrupted'],
    num_rows: 5000
})

In [12]:
def tokenize_dataset(datas):
  inputs=tokenizer(datas['corrupted'],padding=True,max_length=70,truncation=True,return_tensors='tf')
  targets=tokenizer(datas['sentence'],padding=True,max_length=70,truncation=True,return_tensors='tf')
  input_ids=inputs.input_ids
  attention_mask=inputs.attention_mask
  labels=targets.input_ids

  return {
      'input_ids':input_ids,
      'attention_mask':attention_mask,
      'labels':labels
  }


In [13]:
train_dataset=train_dataset.map(tokenize_dataset,batched=True)
test_dataset=train_dataset.map(tokenize_dataset,batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [19]:
import tensorflow as tf
from tensorflow import keras

In [20]:
train_dataset

Dataset({
    features: ['sentence', 'corrupted', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [21]:
Train_dataset=train_dataset.remove_columns(['corrupted','sentence'])
Test_dataset=test_dataset.remove_columns(['corrupted','sentence'])

In [25]:
train_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 70), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 70), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 70), dtype=tf.int32, name=None))>

In [23]:
def convert_to_tensorflow(data):
  input_ids=tf.convert_to_tensor(data['input_ids'])
  attention_mask=tf.convert_to_tensor(data['attention_mask'])
  labels=tf.convert_to_tensor(data['labels'])
  dataset=tf.data.Dataset.from_tensor_slices(({
      'input_ids':input_ids,
      'attention_mask':attention_mask,

  },labels))

  return dataset.batch(10)

In [24]:
train_dataset=convert_to_tensorflow(Train_dataset)
test_dataset=convert_to_tensorflow(Test_dataset)

In [26]:
for i in train_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(10, 70), dtype=int32, numpy=
array([[   104,   3912,   1125,  10561,   4725,  10921,   1358,    105,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0],
       [   104,   7754,    453,   4277,  58933,   1216, 122222,  74902,
          1343,  25032,   1125, 180201,  97210,   5974,   3348,  19830,
          4566,   3031,  21253,   5286,  18688,  28414,  22501, 106538,
          1659,  58485,   1125,   1318,    492,    105,      0,      0,
             0, 

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [27]:
from transformers import AdamWeightDecay

In [28]:
tf.config.run_functions_eagerly(True)

In [29]:
model.compile(
    optimizer=AdamWeightDecay(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.summary()

Model: "tf_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  236965632 
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  152700581 
                                                                 
Total params: 237755045 (906.96 MB)
Trainable params: 237755045 (906.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
test_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 70), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 70), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 70), dtype=tf.int32, name=None))>

In [31]:
model.fit(train_dataset,validation_data=test_dataset,epochs=5)

Epoch 1/5
500/500 [==============================] - 737s 1s/step - loss: 1.2090 - accuracy: 0.7865 - val_loss: 0.6236 - val_accuracy: 0.8316
Epoch 2/5
500/500 [==============================] - 725s 1s/step - loss: 0.6534 - accuracy: 0.8355 - val_loss: 0.4820 - val_accuracy: 0.8650
Epoch 3/5
500/500 [==============================] - 723s 1s/step - loss: 2.1437 - accuracy: 0.7383 - val_loss: 2.0478 - val_accuracy: 0.7049
Epoch 4/5
500/500 [==============================] - 818s 2s/step - loss: 0.5600 - accuracy: 0.8670 - val_loss: 0.3618 - val_accuracy: 0.8923
Epoch 5/5
500/500 [==============================] - 740s 1s/step - loss: 0.3855 - accuracy: 0.8951 - val_loss: 0.2635 - val_accuracy: 0.9220


In [33]:
sentence="नेपाल को निकासी व्यापारमा ऊनी गलैंचाले बृद्धि भयको छ।"
inputs=tokenizer([sentence],padding=True,max_length=156,truncation=True,return_tensors='tf')
print(inputs)
output=model(**inputs)
print(output)

{'input_ids': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=
array([[   104,   1795,   1125,  67889, 120715, 166736,    433,  81800,
          4191,   1234,  89870,  27062,   1153,    437,    492,    105]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 16), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
TFMaskedLMOutput(loss=None, logits=<tf.Tensor: shape=(1, 16, 197285), dtype=float32, numpy=
array([[[ -9.554537  , -31.422098  , -30.686073  , ..., -33.075077  ,
         -30.524683  , -34.200798  ],
        [  4.1993995 , -10.585474  ,  -8.817015  , ...,  -5.5677752 ,
          -0.43051958,  -6.1800013 ],
        [ 10.111351  ,  -1.5781741 ,   1.1540027 , ...,  -0.30852938,
           1.3590989 ,   2.1230872 ],
        ...,
        [  2.5924726 , -18.25394   , -15.762308  , ..., -14.768478  ,

In [36]:
import numpy as np

In [37]:
predicted_ids = np.argmax(output.logits, axis=-1)

print(tokenizer.decode(predicted_ids[0],skip_special_tokens=True))

नेपालको निकासी व्यापारमा व्यापारमा गलैंचा बृद्धि बृद्धि भयको छ ।
